# Proyecto Sistemas Computacionales A-2018: Preprocesamiento del dataset
---

Los datos recolectados mediante el [formulario](https://goo.gl/forms/1p8IwDXPxwXGKYq92) de Google Forms necesitan de un procesamiento previo, lo cual pasa por las siguientes etapas:



1.   **Extracción de datos de interés**: en la encuesta también se recolectaron datos para un proyecto diferente, por lo cual se deben extraer únicamente los datos de interes, los cuales corresponden a las preguntas 4, 5 y 6.
2.   **Limpieza y tokenización**: se deben remover aquellas palabras sin un significado semántico específico para los sentimientos a tomar en cuenta.
3.   **Representación vectorial del vocabulario**: las oraciones deben ser representadas como un vector *n-* dimensional, por lo cual primero se debe encontrar la representación vectorial de cada palabra. Para esto se hará uso del modelo [Word2Vec](https://en.wikipedia.org/wiki/Word2vec).
4.   **Representación vectorial de las oraciones**: Una vez obtenida la representación vectorial de las palabras, se puede obtener la representación vectorial de las oraciones llevando un procedimiento que combine los vectores de cada palabra contenida en la oración, lo cual puede ser un promedio *coordenada-a-coordenada*. En este caso, se utilizará un promedio ponderado, con un peso conocido como [tf-idf](https://es.wikipedia.org/wiki/Tf-idf).


## Instalación de algunos módulos requeridos
---



In [1]:
!pip install gensim  # For the Word2Vec model
!pip install tqdm    # Just for using a progress bar
!pip install bokeh   # For graphs

    100% |████████████████████████████████| 22.6MB 1.0MB/s 
    100% |████████████████████████████████| 1.4MB 10.8MB/s 
    100% |████████████████████████████████| 133kB 20.1MB/s 
    100% |████████████████████████████████| 4.2MB 6.6MB/s 
    100% |████████████████████████████████| 61kB 18.4MB/s 
    100% |████████████████████████████████| 552kB 22.5MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /content/.cache/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file
    100% |████████████████████████████████| 51kB 3.5MB/s 


    100% |████████████████████████████████| 14.7MB 1.9MB/s 
  Running setup.py bdist_wheel for bokeh ... - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/ff/28/51/22e8d08e9d5383ee1de981aaa8ff7bc53c7d65022e5101400f
Successfully built bokeh


## Importación de todos los módulos requeridos
---


In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
from string import punctuation
from random import shuffle
import io
import csv

import gensim
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import nltk

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from google.colab import files

# importing bokeh library for interactive dataviz
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

pd.options.mode.chained_assignment = None
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Subida de dataset al servidor
---

Este notebook fue inicialmente creado para trabajar en Google Colaboratory, por lo cual es necesario subir el archivo del dataset a sus servidores. El dataset puede ser encontrado [aquí](https://drive.google.com/open?id=1ib9bswNfSrqHCiyQyFDhEDPNKwuSjV4y).

In [3]:
uploaded = files.upload()

Saving labeled_training_data.csv to labeled_training_data.csv


## Carga del dataset
---

El siguiente paso es, entonces, cargar el data set, en este caso en un DataFrame de Pandas. Sin embargo, directamente mientras se realiza la ingesta, se realizará la limpieza de los datos

### Limpieza y tokenización del dataset

Para esto se crea la función `clean_sentence`, la cual realiza lo siguiente:
. Simultaneamente, se realiza la tokenización de las oraciones en palabras, utilizando el módulo **simple_preprocess**, de **gensim**, el cual, además, realiza otras limpiezas básicas, como la eliminación de signos de puntuación:

*   Utiliza el corpus en español de **stopwords** o palabras vacías provisto por **nltk** para eliminar las palabras sin significado semántico importante.
*  Simultaneamente, realiza la tokenización de las oraciones en palabras, utilizando el módulo `simple_preprocess`, de **gensim**, el cual, además, realiza otras limpiezas básicas, como la eliminación de signos de puntuación.



In [0]:
stopWords = nltk.corpus.stopwords.words('spanish')

def clean_sentence(sentence):
    tokens = [word for word in simple_preprocess(sentence)
              if word not in stopWords]
    
    return tokens 
        

### Carga o ingesta del dataset al DataFrame

Esta tarea la realiza la función `ingest`, la cual carga, limpia y tokeniza el dataset, en formato csv, en un DataFrame de Pandas:

In [5]:
def ingest(datasetFileName):    
    data = pd.read_csv(datasetFileName, header = None)
    data.columns = ['sentence', 'sentiment']
    data['sentiment'] = data['sentiment'].map({
                                                'positivo': 1,
                                                'neutral': 0,
                                                'negativo': -1
                                              })
    data['tokens'] = data['sentence'].progress_map(clean_sentence)
    data.reset_index(inplace = True)
    data.drop('index', axis=1, inplace=True)
    print 'dataset loaded with shape', data.shape    
    return data

data = ingest('labeled_training_data.csv')
data.head(5)

progress-bar: 100%|██████████| 186/186 [00:00<00:00, 9437.26it/s]

dataset loaded with shape (186, 3)


,sentence,sentiment,tokens
0,Me siento muy afortunado de tenerte aquí justo...,1,"[siento, afortunado, tenerte, aquí, justo, lado]"
1,Odio a aquellos profesores que creen sabérsela...,-1,"[odio, aquellos, profesores, creen, sabérselas..."
2,El día de hoy se puede ir a hacer compras en e...,0,"[día, hoy, puede, ir, hacer, compras, supermer..."
3,"Hoy quiero conquistar el mundo, tengo mucha en...",1,"[hoy, quiero, conquistar, mundo, mucha, energí..."
4,"Ya no vale la pena seguir intentando, puesto q...",-1,"[vale, pena, seguir, intentando, puesto, enten..."


## Creación del modelo Word2Vec
---

La implementación a utilizar será la de **gensim**. Sin embargo, primero se debe dividir el dataset en conjuntos de entrenamiento y prueba, con relación 7:3:

In [0]:
trainSentences, testSentences, trainLabels, testLabels = \
    train_test_split(np.array(data.tokens),
                     np.array(data.sentiment), test_size = 0.3)

El modelo Word2Vec, internamente, utiliza una red neuronal de 2 capas, mas sin embargo, no es necesario tener conocimiento de esto para su uso (más información [aquí](https://en.wikipedia.org/wiki/Word2vec) y [aquí](https://www.tensorflow.org/tutorials/word2vec)). Lo importante es indicar al modelo la dimensión de los vectores que representarán las palabras y otros parámetros importantes:

### Dimensión de los vectores

La calidad de la vectorización depende y aumenta a media que la dimensión de los vectores aumenta; sim embargo, el incremento marginal de la mejora disminuye en cierto punto. Los valores típicos de la dimensión están en el intervalo [100, 1000]. En este caso, se tomará una dimensión de 200, pero debe evaluarse este y otros parámetros para la optimización del modelo.

### Ventana contextual

Esto determina la cantidad de palabras a la izquierda y derecha que se toman como contexto de una palabra. Se debe realizar un estudio de la longitud promedio de las oraciones, pero, como en la encuesta se pidió un mínimo de 8 palabras, quitando las palabras vacías, se utilizará, por ahora, una ventana de 10, de modo que todas las palabras de la oración entren en el contexto.

In [14]:
vectorDimension = 200
wordsModel = Word2Vec(trainSentences, size = vectorDimension, min_count = 2,
                      window = 10)
wordsModel.train([train for train in tqdm(trainSentences)],
                 total_examples = len(trainSentences),
                 epochs = 10)

100%|██████████| 130/130 [00:00<00:00, 43360.60it/s]


(1047, 7050)

### Obtención de la representación vectorial de palabras

In [15]:
wordsModel['bien']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 1.04287907e-03,  6.83641178e-04, -1.84986013e-04, -3.63560248e-04,
        2.07867869e-03, -5.36761770e-04, -1.62461132e-03,  1.87407903e-04,
       -1.90005277e-03, -2.11084331e-03, -5.92263357e-04, -2.37121596e-03,
        2.32676649e-03,  3.12498421e-04, -2.06115725e-03,  2.31368485e-05,
       -1.00818439e-03,  1.29870721e-03, -8.52750265e-04,  1.53166242e-03,
       -4.24570200e-04,  8.11400707e-04, -1.75629699e-04, -2.28275880e-04,
        2.16728752e-03,  1.77669630e-04, -5.04668933e-05, -1.90309016e-03,
       -1.20362546e-03,  6.97842741e-04,  2.04553988e-04,  1.02920632e-03,
        7.28196756e-04, -2.20082444e-03,  1.83546834e-03, -2.00586161e-03,
        1.30551087e-03,  2.45204964e-03, -1.45027565e-03, -1.05389324e-03,
        1.17183430e-03, -2.56560190e-04, -3.18965904e-04,  4.84990771e-04,
        9.26648863e-06,  2.43824883e-03, -1.88390922e-03, -9.67789907e-04,
       -1.88285869e-03,  9.80376615e-04,  1.42556339e-04, -2.29587918e-03,
        9.08730435e-05,  

Como se observa, el vector tiene dimensión 200.

### Obtención de las palabras más similares

In [16]:
wordsModel.most_similar('bien')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[(u'estudiar', 0.1869836151599884),
 (u'cada', 0.1651230901479721),
 (u'hacer', 0.12186288833618164),
 (u'ayer', 0.1079261377453804),
 (u'bueno', 0.09922540932893753),
 (u'dolor', 0.08765514194965363),
 (u'momento', 0.08533925563097),
 (u'ejercicio', 0.07750966399908066),
 (u'pasan', 0.07586630433797836),
 (u'pasar', 0.07418304681777954)]

Es muy notable que se necesita un corpus más grande de palabras para el uso de este tipo de modelos

## Validación gráfica del modelo Word2Vec
---

Para esto, se disminuye la dimensión de los vectores de las palabras a 2 para poder observar su gráfico 2D.

In [17]:
# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width = 700, plot_height = 600,
                       title = 'Map of word vectors',
                       tools = 'pan, wheel_zoom, box_zoom, reset, hover, '
                               'previewsave',
                       x_axis_type = None, y_axis_type = None, min_border = 1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [wordsModel[w] for w in wordsModel.wv.vocab.keys()[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors

tsne_model = TSNE(n_components = 2, verbose = 1, random_state = 0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns = ['x', 'y'])
tsne_df['words'] = wordsModel.wv.vocab.keys()[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x = 'x', y = 'y', source = tsne_df)
hover = plot_tfidf.select(dict(type = HoverTool))
hover.tooltips = {"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


[t-SNE] Computing 89 nearest neighbors...
[t-SNE] Indexed 90 samples in 0.000s...
[t-SNE] Computed neighbors for 90 samples in 0.006s...
[t-SNE] Computed conditional probabilities for sample 90 / 90
[t-SNE] Mean sigma: 0.006239
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.788898
[t-SNE] Error after 1000 iterations: 0.811682


Se observa, de esta manera, las aglomeración de palabras similares en la nube obtenida. Sin embargo, como ya se dijo, se necesita un vocabulario más grande y oraciones más largas.

## Representación vectorial de oraciones
---

Es hora entonces de obtener la representación vectorial de las oraciones, lo cual termina de preparar la data para ser alimentada al clasificador a utilizar. Como se dijo anteriormente, se utilizará un promedio ponderado de los vectores de cada palabra en la oración. El peso en cuestión será obtenido utilizando la métrica [Tf-idf](https://es.wikipedia.org/wiki/Tf-idf), lo cual, a grandes rasgos, representa la importancia de una palabra en la oración.

### Creación de la matriz de pesos Tf-idf

Esto será llevado a cabo con el módulo `TfidfVectorizer` de **sklearn**:

In [43]:
print 'building tf-idf matrix ...'
vectorizer = TfidfVectorizer(analyzer = lambda x: x)
matrix = vectorizer.fit_transform([sentence
                                   for sentence in tqdm(trainSentences)])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print 'vocab size :', len(tfidf)

100%|██████████| 130/130 [00:00<00:00, 75541.63it/s]

building tf-idf matrix ...
vocab size : 524


### Conversión oración a vector

La siguiente función, `buildWordVector`, calcula el promedio ponderado de los vectores de las palabras de la oración y construye el vector representativo de la oración.

> **Nota:** es importante notar que no se está manejando el caso de que una palabra de la oración no esté en el vocabulario del modelo Word2Vec, lo cual debe ser estudiado para su manejo.



In [0]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += wordsModel[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

Se finaliza, entonces, con la conversión de oración a vectores de todas las oraciones de los corpus de entrenamiento y prueba:

In [48]:
trainSentencesVectors = np.concatenate([buildWordVector(w, vectorDimension)
                                        for w in tqdm(trainSentences)])
trainSentencesVectors = scale(trainSentencesVectors)

testSentencesVectors = np.concatenate([buildWordVector(w, vectorDimension)
                                       for w in tqdm(testSentences)])
testSentencesVectors = scale(testSentencesVectors)

  0%|          | 0/130 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
100%|██████████| 56/56 [00:00<00:00, 5607.36it/s]


Una pequeña y muy simple validación: el tamaño de la lista de oraciones de cada corpus debe ser igual al tamaño de su correspondiente lista de vectores:

In [47]:
print 'Longitud de la lista de oraciones de entrenamiento:', \
      len(trainSentences)
print 'Longitud de la lista de vectores de entrenamiento:', \
      len(trainSentencesVectors)
print 'Longitud de la lista de oraciones de prueba:', \
      len(testSentences)
print 'Longitud de la lista de vectores de prueba:', \
      len(testSentencesVectors)
    
trainSentencesVectors[0]

Longitud de la lista de oraciones de entrenamiento: 130
Longitud de la lista de vectores de entrenamiento: 130
Longitud de la lista de oraciones de prueba: 56
Longitud de la lista de vectores de prueba: 56


array([-1.65520097,  0.79132586, -1.33230847,  1.38317886,  0.84141921,
        0.96656358,  1.17393864, -0.21622531,  2.66983167,  1.22714443,
       -1.96574413, -1.96512581, -1.95821358,  1.36769148, -0.59899523,
       -0.0031911 , -0.48036482,  0.17773873, -1.90426324, -0.86367943,
        2.01926898,  1.96798257,  2.06952563,  1.86806703, -0.49970151,
       -2.00259324, -1.43563706, -2.58463137,  0.32268911,  0.57283145,
       -0.04632699,  1.83465151,  0.18081076, -2.18363916, -1.41367904,
       -1.58799429,  1.67979231,  0.55173245,  2.49723177, -2.30250143,
        2.07075508,  2.72699485,  0.61354736, -2.0450455 , -2.79627991,
       -1.44606226,  1.71068665,  1.26232007,  2.1841695 , -0.0377038 ,
       -1.43257717, -0.7883588 ,  0.74355305, -0.12999519,  0.79657741,
       -2.63503118, -0.41426828, -0.81098841,  2.28193016, -2.36639639,
       -0.09099696,  0.94550517,  1.94943066,  1.21719032, -2.72051448,
       -0.83165874, -1.3953293 , -1.24257782, -1.11001372,  2.28

## Trabajo por realizar
---

*   Es necesario estudiar las formas de **validación** de este tipo de modelos pues, al final del día, es un método de aprendizaje automático no supervisado y debe ser validado y potencialmente **optimizado**.
*   Se deben estudiar y resolver aspectos importantes, como el manejo de oraciones con palabras que no existan en el vocabulario de entrenamiento.



